In [ ]:
import cv2
import os
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)  # Only detect ONE hand
mp_drawing = mp.solutions.drawing_utils

# Create dataset directory
dataset_path = "KENYA_SIGN_LANGUAGE/alphabet_dataset"
os.makedirs(dataset_path, exist_ok=True)

# Define letters (A-z)
letters = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
#print(letters)

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

cv2.namedWindow("KSL Data Collector", cv2.WINDOW_NORMAL)
cv2.resizeWindow("KSL Data Collector", 800, 600)

for letter in letters:
    letter_path = os.path.join(dataset_path, letter)
    os.makedirs(letter_path, exist_ok=True)
    
    print(f"\nCollecting images for letter: {letter}")
    print("Press:")
    print("  's' - Save when hand is properly boxed")
    print("  'q' - Finish this letter")
    
    count = len(os.listdir(letter_path))
    target_images = 30  # Adjust as needed

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame")
            break
            
        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Convert to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        # Clear instructions
        instruction = f"Show ONE hand for: {letter} ({count}/{target_images})"
        cv2.putText(frame, instruction, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        
        # Hand detection and boxing
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks (optional)
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Get hand bounding box
                h, w, _ = frame.shape
                x_max = y_max = 0
                x_min, y_min = w, h
                
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    if x > x_max: x_max = x
                    if x < x_min: x_min = x
                    if y > y_max: y_max = y
                    if y < y_min: y_min = y
                
                # Add padding and draw box
                padding = 20
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,0), 2)
                
                # Display "READY" when hand is detected
                cv2.putText(frame, "READY TO CAPTURE", (x_min, y_min-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
        
        cv2.imshow("KSL Data Collector", frame)
        
        key = cv2.waitKey(1)
        
        if key == ord('s') and results.multi_hand_landmarks:
            # Crop and save only the boxed hand region
            hand_crop = frame[y_min:y_max, x_min:x_max]
            if hand_crop.size != 0:
                img_path = os.path.join(letter_path, f"{letter}_{count}.jpg")
                cv2.imwrite(img_path, hand_crop)
                print(f"Saved: {img_path}")
                count += 1
                
                # Show confirmation
                frame[y_min:y_max, x_min:x_max] = cv2.blur(frame[y_min:y_max, x_min:x_max], (30,30))
                cv2.putText(frame, "SAVED!", (w//2-100, h//2), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
                cv2.imshow("KSL Data Collector", frame)
                cv2.waitKey(300)
                
                if count >= target_images:
                    break
                    
        elif key == ord('q'):
            break
        elif key == 27:  # ESC
            hands.close()
            cap.release()
            cv2.destroyAllWindows()
            exit()

print("\nData collection complete!")
hands.close()
cap.release()
cv2.destroyAllWindows()

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Config
dataset_path = "KENYA_SIGN_LANGUAGE/alphabet_dataset"  # Updated path
img_size = 256
test_ratio = 0.1
val_ratio = 0.2

# A–Z letters
letters = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
letter_to_label = {letter: idx for idx, letter in enumerate(letters)}

# Load and preprocess images
X = []
y = []

print("Loading and preprocessing images...")

for letter in letters:
    letter_folder = os.path.join(dataset_path, letter)
    if not os.path.exists(letter_folder):
        print(f"Warning: Folder {letter_folder} not found. Skipping.")
        continue

    for img_name in os.listdir(letter_folder):
        img_path = os.path.join(letter_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  # Skip unreadable files

        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0  # Normalize
        X.append(img)
        y.append(letter_to_label[letter])

X = np.array(X, dtype=np.float32)
y = np.array(y)

print(f"Total samples loaded: {len(X)}")

# Split into train, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_ratio, stratify=y, random_state=42)
val_size = val_ratio / (1 - test_ratio)  # Adjust val ratio from remaining
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size, stratify=y_temp, random_state=42)

print(f"Split sizes => Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# Save to .npz files
np.savez_compressed("KENYA_SIGN_LANGUAGE/ksl_alphabet_train.npz", X=X_train, y=y_train)
np.savez_compressed("KENYA_SIGN_LANGUAGE/ksl_alphabet_val.npz", X=X_val, y=y_val)
np.savez_compressed("KENYA_SIGN_LANGUAGE/ksl_alphabet_test.npz", X=X_test, y=y_test)

print("Datasets saved successfully: train, val, test")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Dataset path
data_path = "KENYA_SIGN_LANGUAGE"

letters = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
letter_to_label = {letter: idx for idx, letter in enumerate(letters)}
num_classes = 26

# Load preprocessed data
train_data = np.load(os.path.join(data_path, "ksl_alphabet_train.npz"))
val_data = np.load(os.path.join(data_path, "ksl_alphabet_val.npz"))
test_data = np.load(os.path.join(data_path, "ksl_alphabet_test.npz"))

X_train, y_train = train_data["X"], to_categorical(train_data["y"], num_classes=num_classes)
X_val, y_val = val_data["X"], to_categorical(val_data["y"], num_classes=num_classes)
X_test, y_test = test_data["X"], to_categorical(test_data["y"], num_classes=num_classes)

print("Datasets loaded:")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# CNN model 
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="softmax")
])

# Print model summary
model.summary()

# Compile model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',     # You can also monitor 'val_accuracy'
    patience=5,             # Number of epochs with no improvement to wait
    restore_best_weights=True,  # Restore model from best epoch
    verbose=1
)

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=60,
    batch_size=32,
    callbacks=[early_stopping]  # 👈 Add the callback here
)
# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.2f}")

# Save model
model_path = os.path.join(data_path, "ksl_alphabet_model.h5")
model.save(model_path)
print(f"Model saved to: {model_path}")

# -------------------------------------------
# 📊 Plot training and validation accuracy/loss
# -------------------------------------------
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

# -------------------------------------------
# 📉 Confusion Matrix
# -------------------------------------------
# Predict classes on test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[chr(i) for i in range(65, 91)])
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Confusion Matrix (Test Set)")
plt.show()


In [1]:
#exec

import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Load trained model (expects 256x256 input)
model = tf.keras.models.load_model("KENYA_SIGN_LANGUAGE/ksl_alphabet_model.h5")

# Class labels (A-Z)
letters = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
num_classes = 26

# MediaPipe hand setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Webcam setup
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Cannot access webcam")
    exit()

cv2.namedWindow("KSL Alphabet Predictor", cv2.WINDOW_NORMAL)
cv2.resizeWindow("KSL Alphabet Predictor", 800, 600)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    h, w, _ = frame.shape
    prediction = "..."

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Calculate bounding box around the hand
            x_min, y_min = w, h
            x_max = y_max = 0
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)

            padding = 20
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)

            hand_img = frame[y_min:y_max, x_min:x_max]
            if hand_img.size == 0:
                continue

            # Preprocess hand image: resize to (256,256), normalize
            img = cv2.resize(hand_img, (256, 256))
            img = img / 255.0
            img = np.expand_dims(img, axis=0)

            # Predict
            pred = model.predict(img)
            class_id = np.argmax(pred)
            confidence = pred[0][class_id]

            prediction = f"{letters[class_id]} ({confidence*100:.1f}%)"

            # Draw bounding box and label
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, prediction, (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.imshow("KSL Alphabet Predictor", frame)

    key = cv2.waitKey(1)
    if key == 27:  # ESC key to quit
        break

cap.release()
cv2.destroyAllWindows()
hands.close()



2025-06-04 16:49:32.884132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749044972.920489   17639 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749044972.931753   17639 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749044972.960715   17639 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749044972.960746   17639 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749044972.960749   17639 computation_placer.cc:177] computation placer alr

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'KENYA_SIGN_LANGUAGE/ksl_alphabet_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)